In [ ]:
import pandas as pd
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_precision,
    context_recall
)
from ragas import evaluate

In [ ]:
so sánh gt vs res
so sánh retrieved docs vs res

In [5]:
path = "lich_su_dang.csv"
load_csv = pd.read_csv(path)
load_csv.head()

,question,ground_truth_answer
0,Chiến lược chung của ĐCSVN trong cuộc Tổng khở...,Đảng lãnh đạo nhân dân thực hiện Tổng khởi ngh...
1,Hội Việt Nam Cách mạng Thanh niên (1925) do Ng...,Hội Việt Nam Cách mạng Thanh niên ra đời năm 1...
2,"Phong trào ""vô sản hóa"" (từ 1928) trong công n...","Phong trào ""vô sản hóa"" được Hội Việt Nam Cách..."
3,Hội Việt Nam Cách mạng Thanh niên đã tiến hành...,Hội đã xuất bản báo Thanh Niên (xuất bản ở Tru...
4,Cuối năm 1929 có những tổ chức cộng sản nào đư...,"Cuối năm 1929, tại các vùng Việt Nam lần lượt ..."


In [52]:
eval_questions = load_csv['question'].tolist()
eval_answers = load_csv['ground_truth_answer'].tolist()

In [8]:
eval_samples = [
    {"query": q, "ground_truth": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [40]:
# Alternative import method if above doesn't work
import sys
sys.path.append('..')


from retriever.parent_retrieval import parent_document_search
import agent


In [58]:
# Fix: parent_document_search returns a list, not a dict
contexts = []
for q in eval_questions:
    results = parent_document_search(q, 'lich-su-dang-children', 'lich-su-dang')
    contx = []
    if results:  # Check if results is not empty
        for res in results:
            # Get content from each result
            content = res['parent_chunk']['metadata']['content']
            contx.append(content)
        # Add the combined context for this question
        contexts.append(contx)
    else:
        contexts.append([])  # Empty list if no results
        
print(f"Found {len(contexts)} contexts (should be {len(eval_questions)})")
print(f"First context has {len(contexts[0])} documents")
if contexts[0]:
    print(f"First document length: {len(contexts[0][0])}")
# answers = [agent.generate_answer(q, c, agent.get_router(q), None, "gemini-2.5-pro") for q, c in zip(eval_questions, contexts)]

Found 70 contexts (should be 70)
First context has 5 documents
First document length: 1996


In [57]:
len(eval_questions)

70

In [46]:
# Debug parent_document_search function
import retriever.cache_data as cache_data

# Kiểm tra kết nối Pinecone
dense_index, sparse_index = cache_data.get_pinecone_index()

# Kiểm tra namespaces có sẵn
print("Checking namespaces...")
child_stats = dense_index.describe_index_stats()
print("Available namespaces:", list(child_stats.namespaces.keys()))

Checking namespaces...
Available namespaces: ['lich-su-dang-children', 'triet-hoc', 'triet-hoc-children', 'lich-su-dang']
